In [ ]:
# from tqdm import tqdm
# import numpy as np
# from collections import Counter
# from threading import Thread

# import inverted_index_gcp
import inverted_index_gcp
import math
import pickle
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import time
from inverted_index_gcp import MultiFileReader, InvertedIndex
import struct
import time
import os
from google.cloud import storage
from contextlib import closing
import pickle as pkl

import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'myfirstgcp-370210'
!gcloud config set project {project_id}

data_bucket_name = 'elad_318640828'
# try:
#     if os.environ["elad_318640828"] is not None:
#         pass
# except:
#       !mkdir wikidumps
      # !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].


In [ ]:
import pickle

# Replace <path/to/dictionary.pkl> with the path to your dictionary file
with open(f'{data_bucket_name}/postings_gcp/index_body.pkl', 'rb') as f:
    index_body = pickle.load(f)



In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS

In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    print(inverted.posting_locs)
    locs = inverted.posting_locs[w]
    print(locs)
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
# query = "world cUp"
def search_body(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    # query = request.args.get('query', '')
    # if len(query) == 0:
    #   return jsonify(res)
  # BEGIN SOLUTION
    with_stop_words_query_tokens = [token.group() for token in RE_WORD.finditer(query.lower())]

    query_tokens = []
    for tok in with_stop_words_query_tokens:
        if tok not in all_stopwords:
            query_tokens.append(tok)
    query_tokens = Counter(query_tokens)

    # this is normaliztion of the query for the cosine similarity
    norm_factor = 0
    for term, freq in query_tokens.items():
      norm_factor += freq**2
    norm_factor = math.sqrt(norm_factor)
    # print(norm_factor)
    if norm_factor == 0:
      raise ValueError("you insert an empty query")

    sim_dict = {}
    for term, freq in query_tokens.items():
      try:
        posting_list_per_term = read_posting_list(index_body, term)
        # print("-------")
        # print(posting_list_per_term)
        # print("-------")
      except:
        continue
      norm_factor_doc = 0
      for doc_id, score in posting_list_per_term:
        if doc_id in sim_dict:
          sim_dict[doc_id] += score
          # print(score)
        else:
          sim_dict[doc_id] = score
      for doc_id in sim_dict.keys():
        norm_factor_doc += sim_dict[doc_id]**2
      norm_factor_docs_final = math.sqrt(norm_factor_doc)
      if norm_factor_docs_final == 0:
        raise ValueError("You need to check your '.bin' files maybe they are not include in the folder")
    similarity_final_dict = dict(sorted(sim_dict.items(), key= lambda x: (x[1]/(norm_factor * norm_factor_docs_final)), reverse= True))

    res = list()
    for doc_id in similarity_final_dict.keys():
      res.append((doc_id, index_body.id2titles[doc_id]))
    print(res)
  # END SOLUTION
    return res



# search_body(query)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[]